In [ ]:
import re, pandas, HTSeq, glob, collections, importlib, random, pickle, sys
import numpy as np
import pprint as pp

import matplotlib.pyplot as plt
import matplotlib.patches as patches

sys.path.append('/Users/dp/pma/')
import sameRiver
from sameRiver import snoUtils
from sameRiver.signal_normalizer import signal_normalizer
#from sameRiver.bedgraphs import bedgraphs
from sameRiver.area import *
importlib.reload(sameRiver.area)
import sameRiver.gapped_area
importlib.reload(sameRiver.gapped_area)

import sameRiver.RNA
import sameRiver.set_of_named_mRNAs
import sameRiver.gapped_area_mRNA

importlib.reload(sameRiver.RNA)
importlib.reload(sameRiver.set_of_named_mRNAs)
importlib.reload(sameRiver.gapped_area_mRNA)

In [ ]:

importlib.reload(sameRiver.RNA)
importlib.reload(sameRiver.set_of_named_mRNAs)
importlib.reload(sameRiver.gapped_area_mRNA)

_mRNAs = sameRiver.set_of_named_mRNAs.set_of_named_mRNAs()

_mRNAs.create_set_of_named_mRNAs(
    gtf_filename='/opt/genomes/repeats/repeats_and_genome.gtf'
    #gtf_filename='/opt/genome/Homo_sapiens.GRCh38.83/exon_CDS_ensembl_havana_only_Homo_sapiens.GRCh38.83_plus_snos_and_scar_from_ensembl_release91_plus_snRNA_this_release.gtf'
                                )


In [ ]:
_mRNAs.keep_only_mRNA()
_mRNAs.example()

In [ ]:
genomic_fasta_fname = '/opt/genome/refseq/GRCh38_latest_genomic.fna'
genomic_fasta_fname = '/opt/genome/Homo_sapiens.GRCh38.83/Homo_sapiens.GRCh38.dna.primary_assembly.fa'
genomic_fasta = dict( (s[1], s[0]) for s in HTSeq.FastaReader(genomic_fasta_fname, raw_iterator=True) )

In [ ]:
rRNA18 = """>NR_003286.4 Homo sapiens RNA, 18S ribosomal N5 (RNA18SN5), ribosomal RNA
TACCTGGTTGATCCTGCCAGTAGCATATGCTTGTCTCAAAGATTAAGCCATGCATGTCTGAGTACGCACG
GCCGGTACAGTGAAACTGCGAATGGCTCATTAAATCAGTTATGGTTCCTTTGGTCGCTCGCTCCTCTCCT
ACTTGGATAACTGTGGTAATTCTAGAGCTAATACATGCCGACGGGCGCTGACCCCCTTCGCGGGGGGGAT
GCGTGCATTTATCAGATCAAAACCAACCCGGTCAGCCCCTCTCCGGCCCCGGCCGGGGGGCGGGCGCCGG
CGGCTTTGGTGACTCTAGATAACCTCGGGCCGATCGCACGCCCCCCGTGGCGGCGACGACCCATTCGAAC
GTCTGCCCTATCAACTTTCGATGGTAGTCGCCGTGCCTACCATGGTGACCACGGGTGACGGGGAATCAGG
GTTCGATTCCGGAGAGGGAGCCTGAGAAACGGCTACCACATCCAAGGAAGGCAGCAGGCGCGCAAATTAC
CCACTCCCGACCCGGGGAGGTAGTGACGAAAAATAACAATACAGGACTCTTTCGAGGCCCTGTAATTGGA
ATGAGTCCACTTTAAATCCTTTAACGAGGATCCATTGGAGGGCAAGTCTGGTGCCAGCAGCCGCGGTAAT
TCCAGCTCCAATAGCGTATATTAAAGTTGCTGCAGTTAAAAAGCTCGTAGTTGGATCTTGGGAGCGGGCG
GGCGGTCCGCCGCGAGGCGAGCCACCGCCCGTCCCCGCCCCTTGCCTCTCGGCGCCCCCTCGATGCTCTT
AGCTGAGTGTCCCGCGGGGCCCGAAGCGTTTACTTTGAAAAAATTAGAGTGTTCAAAGCAGGCCCGAGCC
GCCTGGATACCGCAGCTAGGAATAATGGAATAGGACCGCGGTTCTATTTTGTTGGTTTTCGGAACTGAGG
CCATGATTAAGAGGGACGGCCGGGGGCATTCGTATTGCGCCGCTAGAGGTGAAATTCTTGGACCGGCGCA
AGACGGACCAGAGCGAAAGCATTTGCCAAGAATGTTTTCATTAATCAAGAACGAAAGTCGGAGGTTCGAA
GACGATCAGATACCGTCGTAGTTCCGACCATAAACGATGCCGACCGGCGATGCGGCGGCGTTATTCCCAT
GACCCGCCGGGCAGCTTCCGGGAAACCAAAGTCTTTGGGTTCCGGGGGGAGTATGGTTGCAAAGCTGAAA
CTTAAAGGAATTGACGGAAGGGCACCACCAGGAGTGGAGCCTGCGGCTTAATTTGACTCAACACGGGAAA
CCTCACCCGGCCCGGACACGGACAGGATTGACAGATTGATAGCTCTTTCTCGATTCCGTGGGTGGTGGTG
CATGGCCGTTCTTAGTTGGTGGAGCGATTTGTCTGGTTAATTCCGATAACGAACGAGACTCTGGCATGCT
AACTAGTTACGCGACCCCCGAGCGGTCGGCGTCCCCCAACTTCTTAGAGGGACAAGTGGCGTTCAGCCAC
CCGAGATTGAGCAATAACAGGTCTGTGATGCCCTTAGATGTCCGGGGCTGCACGCGCGCTACACTGACTG
GCTCAGCGTGTGCCTACCCTACGCCGGCAGGCGCGGGTAACCCGTTGAACCCCATTCGTGATGGGGATCG
GGGATTGCAATTATTCCCCATGAACGAGGAATTCCCAGTAAGTGCGGGTCATAAGCTTGCGTTGATTAAG
TCCCTGCCCTTTGTACACACCGCCCGTCGCTACTACCGATTGGATGGTTTAGTGAGGCCCTCGGATCGGC
CCCGCCGGGGTCGGCCCACGGCCCTGGCGGAGCGCTGAGAAGACGGTCGAACTTGACTATCTAGAGGAAG
TAAAAGTCGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTA"""
def read_fasta(seq):
    _dict = {}
    current_seq_name = ''
    for li in seq.split('\n'):
        if li[0] == '>':
            current_seq_name = li[1:]
            _dict[current_seq_name] = ''
        else:
            _dict[current_seq_name] += li.rstrip('\n')
    return _dict
a = read_fasta(rRNA18)
a = list(a.values())[0]
for chrom, seq in genomic_fasta.items():
    print('Searching ', chrom)
    m = re.search(a, seq)
    print(m)

In [ ]:
def rc(_seq):
    _dict = {'A': 'T', 'T':'A', 'C':'G', 'G':'C', 'N':'N'}
    return ''.join([_dict[x.upper()] for x in _seq[::-1]])


def is_orf(_seq, non_atg_ok=False, not_end_in_stop_ok=False, 
           not_divisible_by_three_ok=False):
    if (len(_seq) % 3):
        print("Not div by 3")
        if not not_divisible_by_three_ok:
            return False
           
    if (_seq[:3] != 'ATG'):
        print('Does not start with an ATG')
        if not non_atg_ok:
            return False
        
    if not((_seq[-3:] == 'TAG') or (_seq[-3:] == 'TGA') or (_seq[-3:] == 'TAA')):
        print('Does not end in a stop')
        if not not_end_in_stop_ok:
            return False
        
    return True


def grab_sequence_from_iv_with_offset(iv, fasta_dict, offset=0):
    seq = ''
    if iv.chrom in fasta_dict:
        seq = fasta_dict[iv.chrom][iv.start - 1 + offset:iv.end + offset].upper()
        if iv.strand == '-':
            seq = rc(seq)
    else:
        print("Chrom name {0} not found".format(iv.chrom))
        return ''
    return seq


def read_transcript_sequence(self, fasta_dict):
    # Move to be a method.
    self.area_sequences = []
    self.cds_sequence = ''
    
    _sequences = [] 
    for n, _area in enumerate(self.areas):  # For each genomic iv.
        _sequences.append(grab_sequence_from_iv_with_offset(_area.iv, fasta_dict, offset=0))
    
    return _sequences


def read_exon_sequence(
        self, fasta_dict, offset=0):
    #print('===',)
    done = False
    successful = False
    

    self.exon_seq_list = []
    for n, _exon in enumerate(self.elements['exon']):
        self.exon_seq_list.append(
            grab_sequence_from_iv_with_offset(_exon, fasta_dict, offset=offset))
    self.exon_sequence = ''.join(self.exon_seq_list)
    return True


def read_premRNA_sequence(
        self, fasta_dict, offset=0):
    
    #print(self.name, '---')
    self.premRNA_sequence = ''
    for n, _region in enumerate(self.elements['mRNA']):
    #    print(_region)
        try:
            self.premRNA_sequence += grab_sequence_from_iv_with_offset(
                _region, fasta_dict, offset=offset)
        except:
            print(_region)
    #print('mRNA element:', self.elements['mRNA'])
    #print(self.premRNA_sequence, len(self.premRNA_sequence))
    return True

def read_cds_sequence(
        self, fasta_dict, allow_offset_search=False, offset=0):
    #print('===',)
    done = False
    successful = False
    
    cds_sequence = ''
    
    if not allow_offset_search:
        self.cds_seq_list = []
        for n, _cds in enumerate(self.elements['CDS']):
            self.cds_seq_list.append(
                grab_sequence_from_iv_with_offset(_cds, fasta_dict, offset=offset))
        self.cds_sequence = ''.join(self.cds_seq_list)
        return self.cds_sequence, self.cds_seq_list
    
    offset = 0
    tried_offset = set()
    offset_to_try = set([-2, -1, 0, 1, 2])
    
    while not done:
        cds_sequence, seq_list = read_cds_seqence(self, fasta_dict, allow_offset_search=False,
                                       offset=offset)
        orf = is_orf(cds_sequence)
        if not orf:
            offset = list(offset_to_try - tried_offset)
            if len(offset) == 0:
                print("Could not find acceptible sequence for {0} at any offset.".format(self.name))
                self.cds_sequence, self.cds_seq_list = (cds_sequence, seq_list)
                done = True
            else:
                offset = min(offset)
                tried_offset.add(offset)
                print("Trying offset {0}..,".format(offset))
        else:
            successful = True
            self.cds_sequence, self.cds_seq_list = (cds_sequence, seq_list)
            if (offset != 0):
                print("Found offset sequence at offset {0}".format(offset))
            else:
                pass
                #print("OK sequnce for {0}".format(self.name))  # Found without offset.
            done = True
    #print('+++')
    
    return successful

import random, copy
import sameRiver.gapped_area_mRNA
gaps = {}
success, failure = (set(), set())

for n , (name, _mRNA) in enumerate(_mRNAs.mRNAs.items()):
    gaps[name] = sameRiver.gapped_area_mRNA.gapped_area_mRNA()
    gaps[name].name = name
    gaps[name].elements = copy.copy(_mRNA.elements)
    _mRNA = gaps[name]
    #read_premRNA_sequence(gaps[name], genomic_fasta)
    
    #read_exon_sequence(gaps[name], genomic_fasta)
    
    #if not read_cds_sequence(_mRNA, genomic_fasta):
    #    print("Read {0}/{1} sequences before stopping.".format(n, len(_mRNAs.mRNAs)))
    #    failure.add(_mRNA.name)
    #else:
    #    success.add(_mRNA.name)
    gaps[name].process()
#print(_mRNAs.mRNAs)
print("Failures: {0}".format(failure))
print("# success {0}. # Failure {1}".format(len(success), len(failure)))
    #if n > 1000:
    #    break

In [ ]:
gap_rnas = sameRiver.set_of_named_mRNAs.set_of_named_mRNAs()
gap_rnas.mRNAs = gaps
print(gap_rnas.mRNAs['ENST00000548101'])

In [ ]:
import sameRiver.bedgraphs

def add_mRNA_binding_from_bedgraphs(rnas, bedgraph_list, gene, cutoff=20, norm_len=20):
    """Add normalized signal to a single ga per bedgraph +/- .wig pair."""
    
    print("Reading {}...".format('/Users/dfporter/pma/tables/pvals.xlsx'))
    try:
        targs = pandas.read_excel('/Users/dfporter/pma/tables/pvals.xlsx',
                                 sheet_name=gene)
    except:
        print("Could not open sheet {}. Skipping...".format(gene))
        return signal_normalizer({'left_UTR': 50, 'CDS': 50, 'right_UTR': 50})
    
    print("Translating ENST to gene names...")
    to_do = set(rnas.mRNAs.keys())#[enst_to_name.get(x) for x in rnas.mRNAs])

    print(targs.head())
    targs = [x.split('::')[0] for x in targs.index]
    targs = set([name_to_enst.get(x, x) for x in targs])
    print("Finding overlap...")
    to_do = to_do & targs
    total_rnas = len(to_do)
    
    all_regions = ['left_UTR', 'CDS', 'right_UTR']
    
    # Determine the size of the average regions to normalize to.
    region_average_lens = rnas.average_region_lengths()
    for k, v in region_average_lens.items():
        region_average_lens[k] = int(v/10)
    normalizer = signal_normalizer(region_average_lens)
    
    #normalizer = signal_normalizer({'left_UTR': 50, 'CDS': 50, 'right_UTR': 50})
    normalizer.add_region('Whole area', norm_len)
    for rna in rnas.mRNAs.values():
        rna.zero()
    for n_bedgraph, fname in enumerate(bedgraph_list, start=1):
        
        print("Loading bedgraph {}/{}: {}...".format(
            n_bedgraph, len(bedgraph_list), fname))
        
        ga = sameRiver.bedgraphs.bedgraph(fname).ga
        print(fname)
        #for iv, val in ga[HTSeq.GenomicInterval('1', 0, 1E9, '+')].steps():
        #    print(iv, val)
        #    if random.randint(0, 100) == 1:
        #        break

        for n_rna, name in enumerate(rnas.mRNAs):#to_do):#rnas.mRNAs.values()):
            
            mRNA = rnas.mRNAs[name]
            #print(mRNA.name)
            
            if len(mRNA.areas) == 0:
                continue
            #if mRNA.areas[0].iv.strand == '-':
            #    continue

            if random.randint(0, 50000) == 1:
                print(mRNA.name, "{}/{}".format(n_rna, total_rnas))
  
            #total = sum([sum(_arr) for _arr in mRNA.raw_signal_by_type.values()])

            
            mRNA.add_coverage_from_mRNA(
                ga, normalizer, cutoff=cutoff)  # Normalized.
            
            #if total > 10 and random.randint(0, 100) == 0:
            #    print(normalizer.ga)
                
    normalizer.normalize_areas_by_number_of_arrays_added()
    print([(name, sum(x)) for name, x in normalizer.ga.items()])
    
    normalizer.normalize_regions_by_length()
    print("Number of arrays added: {0}".format(normalizer.added_to_area))
    print([(name, sum(x)) for name, x in normalizer.ga.items()])
    return normalizer


gtf_translation_fname = '/opt/genome/Homo_sapiens.GRCh38.83/enst_transcript_id_name_biotype_map.txt'
transl_df = pandas.read_csv(
    gtf_translation_fname, sep='\t')
enst_to_name = dict(zip(transl_df.transcript_id, transl_df.gene_name))
name_to_enst = dict(zip(transl_df.gene_name, transl_df.transcript_id))
print("Loaded translations...")

df = pandas.read_excel('/Users/dfporter/pma/miseq/meta/scheme.xls')
print(df.head())
gene_bc = dict(zip(df['Gene'].tolist(), df.P6_BC))
plots = []
for gene, p6bc in gene_bc.items():
    skip = False
    for _str in ['AURKA', 'YBX', 'STD', 'hnRNPC', 'FBL', '100', 'PCBP1',
                'SF3B1', '700', 'nknown']:
        if _str in gene:
            skip = True
    
    if skip:
        continue
    print('Gene {}'.format(gene))
    
    in_dir = '/Users/dfporter/pma/miseq/Runs/171210/'
    in_dir = '/Users/dfporter/pma/miseq/meta/beds/read_start/'
    #in_dir = '/Volumes/Seagate/hiseq/170924/sams/consensus/'
    #in_dir = '/Users/dfporter/pma/miseq/Runs/170915/sams/consensus/'

    bedgraph_list = glob.glob(in_dir + '/{prefix}*_+.wig'.format(prefix=p6bc))
    #in_dir = '/Users/dfporter/pma/miseq/Runs/170524/sams/consensus/' # TGCCGA
    #bedgraph_list = glob.glob(in_dir + '/{prefix}*_deletions_+.wig'.format(prefix='TGCCGA'))  # DDX3X 

    #bedgraph_list = glob.glob(in_dir + '/{prefix}*_TCA_+.wig'.format(prefix='24TGAGTG'))  # hnRNPC (?)

    #bedgraph_list = glob.glob(in_dir + '/{prefix}*_deletions_+.wig'.format(prefix='AGCTAG'))  # FBL 
    #bedgraph_list = glob.glob(in_dir + '/{prefix}*_deletions_+.wig'.format(prefix='TAGTTG'))  # CSRP2 

    #bedgraph_list = glob.glob(in_dir + '/{prefix}*_deletions_+.wig'.format(prefix='TAGTTG'))  # DDX3X 

    #bedgraph_list = glob.glob(in_dir + '/{prefix}*_deletions_+.wig'.format(prefix='TTAGGC'))  # RQCD1 
    #bedgraph_list += glob.glob(in_dir + '/{prefix}*_deletions_+.wig'.format(prefix='CCACTC'))  # RQCD1 

    #bedgraph_list = glob.glob(in_dir + '/{prefix}*_deletions_+.wig'.format(prefix='TGAGTG'))  # RQCD1 131L
    #bedgraph_list += glob.glob(in_dir + '/{prefix}*_deletions_+.wig'.format(prefix='GTCGTC'))  # RQCD1 131L

    #bedgraph_list = glob.glob(in_dir + '/{prefix}*_deletions_+.wig'.format(prefix='ATCGTG'))  # PCBP1 in 171210
    #bedgraph_list += glob.glob(in_dir + '/{prefix}*_deletions_+.wig'.format(prefix='ATCGTG'))  # PCBP1 in 171210

    print(bedgraph_list)
    normalizer = add_mRNA_binding_from_bedgraphs(gap_rnas, bedgraph_list, gene)
    plots.append(
        plot_normalized_signal(normalizer, gene))


print('done.')

In [ ]:
def seq_around_point_of_highest_coverage(self, fasta_dict, bedgraph_name='Signal',):
    point = point_of_highest_coverage(self, bedgraph_name=bedgraph_name)
    if point:
        iv_for_seq = self.genomic_point_of_highest_coverage
        iv_for_seq.start -= 5
        iv_for_seq.end += 5
        seq = grab_sequence_from_iv_with_offset(iv_for_seq, fasta_dict=fasta_dict)
        #print(seq)
        self.seq_around_highest_point = seq
        return seq
    else:
        self.seq_around_highest_point = ''
        return ''

    
def grab_sequence_from_iv_with_offset(iv, fasta_dict, offset=0):
    seq = ''
    if iv.chrom in fasta_dict:
        seq = fasta_dict[iv.chrom][iv.start - 1 + offset:iv.end + offset].upper()
        if iv.strand == '-':
            seq = rc(seq)
    else:
        print("Chrom name {0} not found".format(iv.chrom))
        return ''
    return seq
    
    
def point_of_highest_coverage(self, bedgraph_name='Signal',):
    #  Move to be a method.
    if bedgraph_name not in self.bedgraphs:
        print("No {0} in bedgraphs.".format(bedgraph_name))
        self.coverage_array = np.array([0])
        return False
    arr = []
    
    n_steps = 0
    for _, _v in self.bedgraphs['Signal'].steps():
        n_steps += 1
    
    mRNA = self.elements['mRNA'][0]

    for n, (step_iv, value) in enumerate(self.bedgraphs['Signal'][mRNA].steps()):
        if (step_iv.end - step_iv.start) < 1E6:
            arr += [value] * (step_iv.end - step_iv.start)
            
    arr = np.array(arr)  # Replace this with sensible code when we can look up the methods.
    if len(arr) > 0:
        _max = arr.max()
        self.coverage_array = arr
    else:
        self.coverage_array = np.array([])
        _max = 0
        return False
    
    indexes_of_maxes = []
    for n, v in enumerate(arr):
        if v == _max:
            indexes_of_maxes.append(n)
    max_index = random.choice(indexes_of_maxes)
    
    if _max > 0:
        cv = self.bedgraphs['Signal'][mRNA]
        self.genomic_point_of_highest_coverage = cv.iv
        self.genomic_point_of_highest_coverage.start = cv.iv.start + max_index 
        self.genomic_point_of_highest_coverage.end = self.genomic_point_of_highest_coverage.start + 1
        return True
    else:
        return False

for n , (name, _mRNA) in enumerate(_mRNAs.mRNAs.items()):
    #print(_mRNA.__dict__)
    p = seq_around_point_of_highest_coverage(_mRNA, genomic_fasta)

    
seqs = []
for n, (name, _mRNA) in enumerate(_mRNAs.mRNAs.items()):
    if (len(_mRNA.coverage_array) > 0) and max(_mRNA.coverage_array) > 1:
        print(_mRNA.seq_around_highest_point)
        print(_mRNA.genomic_point_of_highest_coverage)

In [ ]:


def item_summary(v):
    q = ''
    def _len(_object):
        try:
            return len(_object)
        except:
            return 'none'
    def _try_float(w):
        try:
            return float(w)
        except:
            return 0.
    if type(v) == type([]):
        #q = "type is list. There are {0} element(s).\n".format(len(v))
        #try:
        q += "Lengths are: " + ", ".join([str(_len(x)) for x in v])
        q += " -> sum= {0}\n".format(sum([_try_float(_len(x)) for x in v]))
        try:
            q += [item_summary(x) for x in v]
            q += '~' * 12 + '\n'
        except:
            pass

        #except:
        #    pass
        q += '\n'
        return q
    elif type(v) == type(''):
        q += "String is {0} chars long.\n".format(len(v))
        return q
    else:
        return ''
def summary(self, return_summary=False, print_summary=True):
    li = '=' * 50 + '\n'
    for k, v in self.__dict__.items():
        li += '-' * 20 + '\n'
        li += ':'.join([self.name, k]) + '\n'
        li += 'First chars: ' + str(v)[:100] + '\n'
        li += item_summary(v)
    if 'bedgraphs' in self.__dict__:
        for k, v in self.bedgraphs.items():
            li += 'Bedgraph "{0}": '.format(k)
            li += str(v.chrom_vectors) + '\n'
            #  Assume only one chromosome.
            _chrom = [x for x in v.chrom_vectors.keys()][0]
            li += "Chrom {0} ".format(_chrom)
            li += "Strand {0}\n".format(self.areas[0].iv.strand)
            li += 'Coverage array length: '
            arr_len = 0
            for exon_n, ex in self.exons.items():
                arr = []
                for iv, coverage in v[ex.iv].steps():
                    arr += [coverage] * (iv.end - iv.start)
                arr_len += len(arr)
            li += "{0}".format(arr_len)
    if print_summary:
        print(li)
for mRNA in _mRNAs.mRNAs.values():
    print(mRNA.__dict__.keys())
    summary(mRNA)

In [ ]:
print(seqs)

In [ ]:
import seaborn as sns

def plot_normalized_signal(normalizer, gene_name='Gene name', plots=[]):
    print("Plotting normalized signal for {}...".format(gene_name))
    
    all_regions = ['left_UTR', 'CDS', 'right_UTR']
    combined = normalizer.put_regions_together(['left_UTR', 'CDS', 'right_UTR'])
    #plot_arr(combined)
    plt.clf()
    fig = plt.figure()
    start = 0
    pal = sns.cubehelix_palette(len(all_regions), start=18, rot=-.5)
    for n, name in enumerate(all_regions):
        print(sum(normalizer.ga[name]))
        region_len = normalizer.lengths[name]
        print(region_len, len(normalizer.ga[name]))

        plt.plot(range(start, start+region_len), normalizer.ga[name], c=pal[n])
        start += region_len
    plt.xlabel('Position in average mRNA (scaled)')
    plt.ylabel('Fraction of hnRNP C reads')
    print ('--')
    print(sum(combined))
    #plt.ylim((0,0.03))
    #plt.xlim(0, 800)
    fig.savefig('figs/normalized_coverage_{}.pdf'.format(gene_name))
    plt.show()
    
    plt.clf()
    plt.close()
    
    plots.append({'x': range(start, start+region_len),
                 'Gene name': gene_name,
                 'y': copy.copy(normalizer.ga)})
    return plots